In [1]:
#coding=utf-8
from pyltp import SentenceSplitter
from pyltp import Segmentor
from pyltp import Postagger
from pyltp import SementicRoleLabeller
from pyltp import NamedEntityRecognizer
from pyltp import Parser
import os
import re
modelPath='C:\Users\CJ\ltp_data'

In [2]:
#分词
def segmentor(sentence='你好，你觉得这个例子从哪里来的？'):
    segmentor = Segmentor()  # 初始化实例
    model="cws.model"
    segmentor.load(os.path.join(modelPath, model))  # 加载模型
    words = segmentor.segment(sentence)  # 分词
    #默认可以这样输出
    print '\t'.join(words)
    # 可以转换成List 输出
    words_list = list(words)
    segmentor.release()  # 释放模型
    return words_list

def posttagger(words):
    postagger = Postagger() # 初始化实例
    model = "pos.model"
    postagger.load(os.path.join(modelPath, model))  # 加载模型
    postags = postagger.postag(words)  # 词性标注
    #for word,tag in zip(words,postags):
     #   print word+'/'+tag
    postagger.release()  # 释放模型
    return postags

#分句，也就是将一片文本分割为独立的句子
def sentence_splitter(sentence='你好，你觉得这个例子从哪里来的？'):
    sents = SentenceSplitter.split(sentence)  # 分句
    clean_s = [x for x in list(sents) if x != ""]
    return clean_s

#命名实体识别
def ner(words, postags):
    recognizer = NamedEntityRecognizer() # 初始化实例
    model = "ner.model"
    recognizer.load(os.path.join(modelPath, model))  # 加载模型
    netags = recognizer.recognize(words, postags)  # 命名实体识别
    for word, ntag in zip(words, netags):
        print word + '/' + ntag,
    recognizer.release()  # 释放模型
    return netags

#依存语义分析
def parse(words, postags):
    parser = Parser() # 初始化实例
    model = "parser.model"
    parser.load(os.path.join(modelPath, model))  # 加载模型
    arcs = parser.parse(words, postags)  # 句法分析
    print "\t".join("%d:%s" % (arc.head, arc.relation) for arc in arcs)
    parser.release()  # 释放模型
    return arcs

#角色标注
def role_label(words, postags, netags, arcs):
    labeller = SementicRoleLabeller() # 初始化实例
    model = "srl"
    labeller.load(os.path.join(modelPath, model))  # 加载模型
    roles = labeller.label(words, postags, netags, arcs)  # 语义角色标注
    for role in roles:
        print role.index, "".join(
            ["%s:(%d,%d)" % (arg.name, arg.range.start, arg.range.end) for arg in role.arguments])
    labeller.release()  # 释放模型

In [75]:
def get_org(sentence):
    words = segmentor(sentence)
    tags = posttagger(words)
    netags = ner(words, tags)
    entities = set()
    str_e =""
    for word,ntag in zip(words, netags):
        if ntag == 'B-Ni':
            str_e = word
        elif ntag == 'I-Ni':
            str_e += word
        elif ntag == 'E-Ni':
            str_e += word
            entities.add(str_e)
        elif ntag == 'S-Ni':
            str_e = word
            entities.add(str_e)
        else:
            str_e = ""
    return entities

In [130]:
#去除句子内的标点和非中文
def get_chinese(line):
    result = re.findall(ur"([\u4e00-\u9fa5]|[0-9])", line.decode('utf-8'))
    un = "".join(result)
    try:
        return unicode.encode(un)
    except:
        return ''
def get_chs(lines):
    result = []
    for l in lines:
        chl = get_chinese(l)
        if chl != '':
            result.append(chl)
    return result
def fp(it):
    for i in it:
        print i

In [134]:
sentence="中石化山东济南平阴第五加油站 车用汽油92（ⅴ） 18 平阴县 平阴县洪范农机加油站 普通柴油0 18 平阴县 平阴县洪范农机加油站 车用汽油92（ⅴ） 19 平阴县 中石化山东济南平阴第3加油站 普通柴油0 19 平阴县 中石化山东济南平阴第3加油站 车用汽油95（ⅴ） 20 平阴县 中石化山东济南平阴第4加油站"
 

In [135]:
se = get_entity_P(sentence)

中石化山东济南平阴第五加油站车用汽油9218平阴县平阴县洪范农机加油站普通柴油018平阴县平阴县洪范农机加油站车用汽油9219平阴县中石化山东济南平阴第3加油站普通柴油019平阴县中石化山东济南平阴第3加油站车用汽油9520平阴县中石化山东济南平阴第4加油站
中石化	山东	济南	平阴	第五	加油站	车	用	汽油	9218	平阴县	平阴县	洪范	农机	加油站	普通	柴油	018	平阴县	平阴县	洪范	农机	加油站	车	用	汽油	9219	平	阴县	中石化	山东	济南	平阴	第3	加油站	普通	柴油	019平阴县	中石化	山东	济南	平阴	第3	加油站	车	用	汽油	9520	平阴县	中石化	山东	济南	平阴	第4	加油站
中石化/O 山东/B-Ns 济南/I-Ns 平阴/E-Ns 第五/O 加油站/O 车/O 用/O 汽油/O 9218/O 平阴县/B-Ns 平阴县/E-Ns 洪范/O 农机/O 加油站/O 普通/O 柴油/O 018/O 平阴县/B-Ns 平阴县/E-Ns 洪范/O 农机/O 加油站/O 车/O 用/O 汽油/O 9219/O 平/O 阴县/B-Ns 中石化/I-Ns 山东/I-Ns 济南/I-Ns 平阴/E-Ns 第3/O 加油站/O 普通/O 柴油/O 019平阴县/B-Ns 中石化/I-Ns 山东/I-Ns 济南/I-Ns 平阴/E-Ns 第3/O 加油站/O 车/O 用/O 汽油/O 9520/O 平阴县/S-Ns 中石化/O 山东/B-Ns 济南/I-Ns 平阴/E-Ns 第4/O 加油站/O


In [131]:
sts = sentence_splitter(sentence)
chs = get_chs(sts)
for c in chs:
    print c

中石化山东济南平阴第5加油站车用汽油9218平阴县平阴县洪范农机加油站普通柴油018平阴县平阴县洪范农机加油站车用汽油9219平阴县中石化山东济南平阴第3加油站普通柴油019平阴县中石化山东济南平阴第3加油站车用汽油9520平阴县中石化山东济南平阴第4加油站


In [115]:
def get_entity_P(sentence):
    sts = sentence_splitter(sentence)
    chs = get_chs(sts)
    se = set()
    for i in chs:
        ch_sentence = get_chinese(i)
        print ch_sentence
        entities = get_org(ch_sentence)
        se = se | entities    
    return se

In [116]:
sente = "原标题：淄博市食品药品监督管理局组织抽检发现3批次不合格样品 14日，记者从淄博市食品药品监督管理局了解到，该局通过组织抽检粮食加工品、饮料、肉制品等3类食品139批次样品，共发现3批次不合格样品。 不合格产品情况为：淄博市五老缘山泉水厂生产的天然山泉水，亚硝酸盐(以NaNO2计)项目超标；淄博中农农业科技股份有限公司生产的桶装天然弱碱性饮用水，耗氧量(以O2计)项目超标；淄博资泉水业有限公司生产的桶装饮用水，铜绿假单胞菌项目超标。"

In [117]:
se = get_entity_P(sente)

原标题淄博市食品药品监督管理局组织抽检发现批次不合格样品日记者从淄博市食品药品监督管理局了解到该局通过组织抽检粮食加工品饮料肉制品等类食品批次样品共发现批次不合格样品
原	标题	淄博市	食品	药品	监督	管理局	组织	抽检	发现	批次	不	合格	样品	日记者	从	淄博市	食品	药品	监督	管理局	了解	到	该局	通过	组织	抽检	粮食	加工品	饮料	肉制品	等	类	食品	批次	样品	共	发现	批次	不	合格	样品
原/O 标题/O 淄博市/B-Ni 食品/I-Ni 药品/I-Ni 监督/I-Ni 管理局/E-Ni 组织/O 抽检/O 发现/O 批次/O 不/O 合格/O 样品/O 日记者/O 从/O 淄博市/B-Ni 食品/I-Ni 药品/I-Ni 监督/I-Ni 管理局/E-Ni 了解/O 到/O 该局/O 通过/O 组织/O 抽检/O 粮食/O 加工品/O 饮料/O 肉制品/O 等/O 类/O 食品/O 批次/O 样品/O 共/O 发现/O 批次/O 不/O 合格/O 样品/O 不合格产品情况为淄博市五老缘山泉水厂生产的天然山泉水亚硝酸盐以计项目超标
不	合格	产品	情况	为	淄博市	五	老	缘山	泉水	厂	生产	的	天然	山泉水	亚硝酸盐	以	计	项目	超标
不/O 合格/O 产品/O 情况/O 为/O 淄博市/S-Ns 五/O 老/O 缘山/B-Ni 泉水/I-Ni 厂/E-Ni 生产/O 的/O 天然/O 山泉水/O 亚硝酸盐/O 以/O 计/O 项目/O 超标/O 淄博中农农业科技股份有限公司生产的桶装天然弱碱性饮用水耗氧量以计项目超标
淄博	中农	农业	科技	股份	有限公司	生产	的	桶装	天然	弱碱性	饮用水	耗氧量	以	计	项目	超标
淄博/B-Ni 中农/I-Ni 农业/I-Ni 科技/I-Ni 股份/I-Ni 有限公司/E-Ni 生产/O 的/O 桶装/O 天然/O 弱碱性/O 饮用水/O 耗氧量/O 以/O 计/O 项目/O 超标/O 淄博资泉水业有限公司生产的桶装饮用水铜绿假单胞菌项目超标
淄博	资泉	水业	有限公司	生产	的	桶装	饮用水	铜	绿	假	单胞菌	项目	超标
淄博/B-Ni 资泉/I-Ni 水业/I-Ni 有限公司/E-Ni 生产/O 的/O 桶装/O 饮用水/O 铜/O 绿/O 假/O 单胞菌/O 项目/O 超标/O


In [119]:
fp(se)

淄博资泉水业有限公司
缘山泉水厂
淄博市食品药品监督管理局
淄博中农农业科技股份有限公司


In [113]:
###分句
sts = sentence_splitter(sentence)
chs = get_chs(sts)
se = set()
for i in chs:
    ch_sentence = get_chinese(i)
    print ch_sentence
    entities = get_org(ch_sentence)
    se = se | entities

中国人寿保险股份有限公司湖南省分公司和讯网消息月日湖南保监局发布湘保监罚号文中国人寿保险股份有限公司湖南分公司电销中心在销售过程中的录音存在欺骗投保人或被保险人隐瞒保险合同重要内容的行为
中国	人寿	保险	股份	有限公司	湖南省	分公司	和	讯网	消息	月日	湖南	保监局	发布	湘保	监罚号	文	中国	人寿	保险	股份	有限公司	湖南	分公司	电销	中心	在	销售	过程	中	的	录音	存在	欺骗	投保人	或	被保险人	隐瞒	保险	合同	重要	内容	的	行为
中国/B-Ni 人寿/I-Ni 保险/I-Ni 股份/I-Ni 有限公司/I-Ni 湖南省/I-Ni 分公司/E-Ni 和/O 讯网/O 消息/O 月日/O 湖南/S-Ns 保监局/O 发布/O 湘保/B-Ni 监罚号/I-Ni 文/I-Ni 中国/I-Ni 人寿/I-Ni 保险/I-Ni 股份/I-Ni 有限公司/I-Ni 湖南/I-Ni 分公司/I-Ni 电销/I-Ni 中心/E-Ni 在/O 销售/O 过程/O 中/O 的/O 录音/O 存在/O 欺骗/O 投保人/O 或/O 被保险人/O 隐瞒/O 保险/O 合同/O 重要/O 内容/O 的/O 行为/O 湖南保监局拟中国人寿保险股份有限公司湖南省分公司消息月日湖南保监局发布湘保监罚号文中国人寿保险股份有限公司湖南分公司电销中心在销售过程中的录音存在欺骗投保人或被保险人隐瞒保险合同重要内容的行为
湖南	保监局	拟	中国	人寿	保险	股份	有限公司	湖南省	分公司	消息	月日	湖南	保监局	发布	湘保	监罚号	文	中国	人寿	保险	股份	有限公司	湖南	分公司	电销	中心	在	销售	过程	中	的	录音	存在	欺骗	投保人	或	被保险人	隐瞒	保险	合同	重要	内容	的	行为
湖南/S-Ns 保监局/O 拟/O 中国/B-Ni 人寿/I-Ni 保险/I-Ni 股份/I-Ni 有限公司/I-Ni 湖南省/I-Ni 分公司/I-Ni 消息/E-Ni 月日/O 湖南/S-Ns 保监局/O 发布/O 湘保/B-Ni 监罚号/I-Ni 文/I-Ni 中国/I-Ni 人寿/I-Ni 保险/I-Ni 股份/I-Ni 有限公司/I-Ni 湖南/I-Ni 分公司/I-Ni 电销/I-Ni 中心/E-Ni 在/O 销售/O 过程/O 中/O 的/O 录音/O 存在/O 欺骗/O 